# 🎭 Emotion Recognition Training on Google Colab

This notebook trains your emotion recognition model using Google Colab's free GPU.


## Step 1: Install Dependencies


In [ ]:
# Install required packages
!pip install ultralytics opencv-python -q


## Step 2: Upload Your Data

**Option A: Upload from your computer**
1. Zip your `data` folder (make sure it contains `train/` and `test/` subfolders)
2. Run the cell below and upload the zip file

**Option B: Upload from Google Drive**
1. Upload your data folder to Google Drive
2. Mount Google Drive (see cell below)


In [ ]:
# Option A: Upload zip file from your computer
from google.colab import files
import zipfile
import os

uploaded = files.upload()  # Upload your data.zip file

# Extract the zip file
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"✅ Extracted {filename}")


In [ ]:
# Option B: Mount Google Drive (if your data is on Drive)
from google.colab import drive
drive.mount('/content/drive')

# If your data is in Drive, create a symlink or copy it
# Example: !cp -r "/content/drive/MyDrive/EmotionRecognition/data" "./data"


## Step 3: Verify Data Structure

Make sure your data folder has the correct structure:
```
data/
  train/
    angry/
    happy/
    sad/
    ...
  test/
    angry/
    happy/
    ...
```


In [ ]:
# Check data structure
import os

if os.path.exists('data'):
    print("📁 Data folder found!")
    print("\nTrain classes:", os.listdir('data/train') if os.path.exists('data/train') else 'Not found')
    print("\nTest classes:", os.listdir('data/test') if os.path.exists('data/test') else 'Not found')
else:
    print("❌ Data folder not found. Please upload your data first.")


## Step 4: Check GPU Availability


In [ ]:
# Check if GPU is available
import torch

if torch.cuda.is_available():
    print(f"✅ GPU Available: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  No GPU detected. Training will be slow on CPU.")
    print("   Go to Runtime > Change runtime type > Hardware accelerator > GPU")


## Step 5: Train the Model

**Basic Training (default settings):**


In [ ]:
# Basic training command
!yolo task=classify mode=train model=yolov8n-cls.pt data=data epochs=50 imgsz=224 batch=32 lr0=0.001 optimizer=Adam


**Advanced Training (tuned hyperparameters):**

Uncomment and modify the command below for better results:


In [ ]:
# Advanced training with tuned hyperparameters
# Adjust these parameters based on your needs:
# - epochs: Number of training rounds (50-100+)
# - batch: Batch size (32-64 for GPU)
# - imgsz: Image size (224, 320)
# - lr0: Learning rate (0.0001-0.01)
# - model: Model size (yolov8n-cls.pt, yolov8s-cls.pt, yolov8m-cls.pt)

!yolo task=classify mode=train \
  model=yolov8s-cls.pt \
  data=data \
  epochs=100 \
  imgsz=224 \
  batch=64 \
  lr0=0.0005 \
  optimizer=AdamW \
  weight_decay=0.0005 \
  warmup_epochs=3 \
  patience=20


## Step 6: View Training Results


In [ ]:
# Display training results
from IPython.display import Image, display
import os

results_path = 'runs/classify/train/results.png'
if os.path.exists(results_path):
    display(Image(results_path))
    print("\n📊 Training curves")
else:
    print("Training results not found. Make sure training has completed.")


In [ ]:
# Display confusion matrix
confusion_path = 'runs/classify/train/confusion_matrix.png'
if os.path.exists(confusion_path):
    display(Image(confusion_path))
    print("\n📈 Confusion Matrix")
else:
    print("Confusion matrix not found.")


## Step 7: Download Trained Model

Download the best model weights to use on your laptop:


In [ ]:
# Find the latest training run
import os
import glob

# Get the most recent training directory
train_dirs = glob.glob('runs/classify/train*')
if train_dirs:
    latest_train = max(train_dirs, key=os.path.getmtime)
    best_model = os.path.join(latest_train, 'weights', 'best.pt')
    
    if os.path.exists(best_model):
        print(f"✅ Found model: {best_model}")
        print(f"   File size: {os.path.getsize(best_model) / 1024 / 1024:.2f} MB")
        
        # Download the model
        from google.colab import files
        files.download(best_model)
        print("\n📥 Model downloaded! Save it to your project folder.")
    else:
        print("❌ Model file not found.")
else:
    print("❌ No training runs found.")


## Step 8: Download All Training Results (Optional)

If you want to download the entire training folder:


In [ ]:
# Zip and download entire training folder
import shutil
from google.colab import files

# Create zip of training results
if os.path.exists('runs/classify/train'):
    shutil.make_archive('training_results', 'zip', 'runs/classify/train')
    files.download('training_results.zip')
    print("✅ Training results downloaded!")
else:
    print("❌ Training folder not found.")


## 📝 Notes

- **GPU Runtime**: Make sure to enable GPU in Runtime > Change runtime type > Hardware accelerator > GPU
- **Session Timeout**: Colab sessions timeout after ~12 hours of inactivity. Save your work!
- **File Persistence**: Files are deleted when the session ends. Download your model before closing!
- **Model Location**: After downloading, place `best.pt` in `runs/classify/train/weights/` on your laptop
- **Data Upload**: If your data is large, consider using Google Drive instead of direct upload
